In [28]:
#nbi:hide_in
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import sys
import numpy as np
from scipy import signal
import scipy.io.wavfile
from scipy.io import wavfile
from matplotlib import pyplot
import sklearn.decomposition
import wave
import simpleaudio as sa

In [29]:
#nbi:hide_in
def getaudio(path):
    fs, data = scipy.io.wavfile.read(path)
    return fs, data

def get_seperated_audio(audio):
    audio1 = getaudio('wav_numbers_trim/%s_numbers.wav'%(audio[0].lower()))
    audio2 = getaudio('wav_numbers_trim/%s_numbers.wav'%(audio[1].lower()))
    
    t1 = audio1[1][:,0]
    t2 = audio2[1][:,0]

    t1 = (t1)/ 2**15
    t2 = (t2)/ 2**15
    
    source1 = 0.3*t1 + 0.7*t2
    source2 = 0.7*t1 + 0.3*t2
    
    data = np.c_[source1, source2]

    fast_ica  = sklearn.decomposition.FastICA( n_components=2  )
    separated = fast_ica.fit_transform( data )
    
    assert np.allclose( data, separated.dot( fast_ica.mixing_.T ) + fast_ica.mean_ )

    max_source, min_source = 1.0, -1.0
    max_result, min_result = max(separated.flatten()), min(separated.flatten())
    
    separated = 2*((separated - np.ones(separated.shape)*min_result)/(max_result - min_result)) + np.ones(separated.shape)*(-1)
    
    fig = pyplot.figure( figsize=(10, 8) )
    fig.canvas.set_window_title( 'Blind Source Separation' )

    ax = fig.add_subplot(221)
    ax.set_title('Source #1')
    ax.set_ylim([-1, 1])
    ax.get_xaxis().set_visible( False )
    pyplot.plot( data[:, 0], color='r' )

    ax = fig.add_subplot(223)
    ax.set_ylim([-1, 1])
    ax.set_title('Source #2')
    ax.get_xaxis().set_visible( False )
    pyplot.plot( data[:, 1], color='r' )

    ax = fig.add_subplot(222)
    ax.set_ylim([-1, 1])
    ax.set_title('Separated #1')
    ax.get_xaxis().set_visible( False )
    pyplot.plot( separated[:, 0], color='g' )

    ax = fig.add_subplot(224)
    ax.set_ylim([-1, 1])
    ax.set_title('Separated #2')
    ax.get_xaxis().set_visible( False )
    pyplot.plot( separated[:, 1], color='g' )
    pyplot.show()
    
    
    scipy.io.wavfile.write('separated_1.wav', audio1[0], np.int16(separated[:, 0]*2**15))
    scipy.io.wavfile.write('separated_2.wav', audio1[0], np.int16(separated[:, 1]*2**15))
    scipy.io.wavfile.write('mixed.wav', audio1[0], np.int16(data*2**15))
    
    return data, separated

def playsongs(a):
    #wave_read = wave.open(a, 'rb')
    wave_obj = sa.WaveObject.from_wave_file(a)
    
    play_obj = wave_obj.play()
    play_obj.wait_done()
    
    

In [30]:
#nbi:hide_in
a = widgets.Checkbox(description = "Arabic",)
b = widgets.Checkbox(description = "English")
c = widgets.Checkbox(description = "Hindi")

def f(a, b, c):
    audio = []
    if a == True:
        audio.append('arabic')
    if b == True:
        audio.append('english')
    if c == True:
        audio.append('hindi')
    
    if len(audio) == 2:
        data, separated = get_seperated_audio(audio)
        
        button = widgets.Button(description='Source - %s'%(audio[0]))
        out = widgets.Output()
        
        def on_button_clicked(_):
            with out:
                out.clear_output()
                playsongs("wav_numbers_trim/%s_numbers.wav"%(audio[0]))
        
        button.on_click(on_button_clicked)
        display(widgets.VBox([button,out]))
        
        button = widgets.Button(description='Source - %s'%(audio[1]))
        out = widgets.Output()
        
        def on_button_clicked(_):
            with out:
                out.clear_output()
                playsongs("wav_numbers_trim/%s_numbers.wav"%(audio[1]))
        
        button.on_click(on_button_clicked)
        display(widgets.VBox([button,out]))
        
        button = widgets.Button(description='Mixed Sounds')
        out = widgets.Output()
        
        def on_button_clicked(_):
            with out:
                out.clear_output()
                playsongs("mixed.wav")
        
        button.on_click(on_button_clicked)
        display(widgets.VBox([button,out]))
        
        button = widgets.Button(description='Separated - 1 Sound')
        out = widgets.Output()
        
        def on_button_clicked(_):
            with out:
                out.clear_output()
                playsongs("separated_1.wav")
        
        button.on_click(on_button_clicked)
        display(widgets.VBox([button,out]))
        
        button = widgets.Button(description='Separated - 2 Sound')
        out = widgets.Output()
        
        def on_button_clicked(_):
            with out:
                out.clear_output()
                playsongs("separated_2.wav")
        
        button.on_click(on_button_clicked)
        display(widgets.VBox([button,out]))
        
    

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
print("Select any two audios")
widgets.HBox([widgets.VBox([a, b, c]), out])

Select any two audios


In [31]:
#nbi:hide_in
from scipy.io import wavfile
def getaudio(path):
    fs, data = wavfile.read(path)
    return fs, data